In [5]:
import clickhouse_connect
from dotenv import load_dotenv
import os

import pandas as pd 

load_dotenv()


True

In [6]:
client = clickhouse_connect.get_client(
    host='127.0.0.1', port=8124,
    username=os.getenv('CH_USER_READ'),
    password=os.getenv('CH_PASS_READ'),
    database=os.getenv('CH_DB', 'fgis')
)

In [7]:
print(client.command("SELECT version()"))

18.16.1


In [8]:
tables = client.query("SHOW TABLES FROM fgis").result_rows
for t in tables:
    print(t[0])

mit_details_raw
mit_search_raw
v_vri_with_type
vri_details
vri_details_raw
vri_mieta
vri_mis
vri_search_raw


In [9]:
def sql(query: str, *, params=None, ch_client=client):
    result = ch_client.query(query, parameters=params)
    return pd.DataFrame(result.result_rows, columns=result.column_names)


Сделаем запрос, что бы посмотреть все поверки приборов, изготавливаемых фирмой National Instruments

In [10]:
df = sql("""
    SELECT *
    FROM v_vri_with_type
    WHERE manufacturers LIKE '%National%'
""")

In [11]:
df.head(5) 

,vri_id,verification_date,valid_date,org_title,mi_mitnumber,mi_mititle,mi_mitype,mi_modification,mi_number,mit_uuid,mit_title,mit_notation,manufacturers
0,2-151957054,2019-07-10 12:00:00,2020-07-09 12:00:00,"АО ""АКТИ-МАСТЕР""",74861-19,Преобразователи напряжения и силы тока цифро-а...,"NI 9263, NI 9264, NI 9265",,1DCBF04,5f43b0e2-bcd4-b099-a286-e2f191587b8b,Преобразователи напряжения и силы тока цифро-а...,"NI 9263, NI 9264, NI 9265","Компания ""National Instruments Corporation"", В..."
1,2-154475413,2019-08-02 12:00:00,2020-08-01 12:00:00,"АО ""АКТИ-МАСТЕР""",68422-17,Мультиметры цифровые модульные,"NI PXIe-4080, NI PXIe-4081, NI PXIe-4082",,1CCC913,753c0237-63a2-9190-dc1c-f372ce8f0fd1,Мультиметры цифровые модульные,"NI PXIe-4080, NI PXIe-4081, NI PXIe-4082","Компания ""National Instruments Malaysia Sdn. B..."
2,2-154475414,2019-08-02 12:00:00,2020-08-01 12:00:00,"АО ""АКТИ-МАСТЕР""",68422-17,Мультиметры цифровые модульные,"NI PXIe-4080, NI PXIe-4081, NI PXIe-4082",,1CB7EE2,753c0237-63a2-9190-dc1c-f372ce8f0fd1,Мультиметры цифровые модульные,"NI PXIe-4080, NI PXIe-4081, NI PXIe-4082","Компания ""National Instruments Malaysia Sdn. B..."
3,2-154475415,2019-08-02 12:00:00,2021-08-01 12:00:00,"АО ""АКТИ-МАСТЕР""",56624-14,Источники питания программируемые модульные,"NI PXIe-4112, NI PXIe-4113",,1CCF7A3,51f8b4d7-b6b7-87b6-2024-de12672ff090,Источники питания программируемые модульные,"NI PXIe-4112, NI PXIe-4113","Компания ""National Instruments Corporation"", В..."
4,2-154475416,2019-08-02 12:00:00,2021-08-01 12:00:00,"АО ""АКТИ-МАСТЕР""",56624-14,Источники питания программируемые модульные,"NI PXIe-4112, NI PXIe-4113",,1CE021D,51f8b4d7-b6b7-87b6-2024-de12672ff090,Источники питания программируемые модульные,"NI PXIe-4112, NI PXIe-4113","Компания ""National Instruments Corporation"", В..."


Посмотрим на таблицу с деталями поверки

In [12]:
vri = sql("""
          SELECT *
          FROM vri_details
          LIMIT 10
          """)
vri.head(5)

,vri_id,mitype_number,mitype_type,mitype_title,mitype_url,manufacture_num,manufacture_year,modification,org_title,sign_cipher,...,valid_date,doc_title,applicable_cert_num,applicable_sign_pass,applicable_sign_mi,brief_indicator,run_id,source_tag,ingest_ts,ver
0,2-51433640,45982-10,,Системы автоматизированные измерительные,,1211001,0,ТЕСТ-9110-VXI-126-6800,"ООО ""VXI-Системы""",ГЖН,...,2014-03-01,нет данных,0003-13,0,0,0,vri-from-mit-20251007-203935,manual,2025-10-07 17:42:11,1759858931
1,2-51433691,45982-10,,Системы автоматизированные измерительные,,1211001,0,ТЕСТ-9110-VXI-126-6800,"ООО ""VXI-Системы""",ГЖН,...,2014-03-01,нет данных,0003-13,0,0,0,vri-from-mit-20251007-203935,manual,2025-10-07 17:42:11,1759858931
2,2-51305772,52694-13,,Системы автоматизированные измерительные,,1211003,0,-,"ООО ""VXI-Системы""",ГЖН,...,2014-03-06,нет данных,0005-13,0,0,0,vri-from-mit-20251007-204818,manual,2025-10-07 18:08:43,1759860523
3,2-51305817,52694-13,,Системы автоматизированные измерительные,,1211003,0,-,"ООО ""VXI-Системы""",ГЖН,...,2014-03-06,нет данных,0005-13,0,0,0,vri-from-mit-20251007-204818,manual,2025-10-07 18:08:43,1759860523
4,2-48350937,45982-10,,Системы автоматизированные измерительные,,1302001,0,ТЕСТ-9110-VXI-100-800,"ООО ""VXI-Системы""",ГЖН,...,2015-05-30,нет данных,0014-13,0,0,0,vri-from-mit-20251007-203935,manual,2025-10-07 17:42:11,1759858931


In [13]:
vri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   vri_id                10 non-null     object        
 1   mitype_number         10 non-null     object        
 2   mitype_type           10 non-null     object        
 3   mitype_title          10 non-null     object        
 4   mitype_url            10 non-null     object        
 5   manufacture_num       10 non-null     object        
 6   manufacture_year      10 non-null     int64         
 7   modification          10 non-null     object        
 8   org_title             10 non-null     object        
 9   sign_cipher           10 non-null     object        
 10  mi_owner              10 non-null     object        
 11  vri_type              10 non-null     object        
 12  vri_type_label        10 non-null     object        
 13  vrf_date              1

Объединим таблицы с результатами поверки для National Instruments и деталями поверки для этих записей

In [14]:
full_df = sql("""
    SELECT 
        vt.vri_id,
        vt.verification_date,
        vt.valid_date,
        vt.org_title,
        vt.mi_mitnumber,
        vt.mi_mititle,
        vt.mi_mitype,
        vt.mi_modification,
        vt.mi_number,
        vt.manufacturers,        
        vd.mi_owner,
        vd.vri_type,
        vd.vri_type_label
    FROM v_vri_with_type AS vt
    LEFT JOIN vri_details as vd ON vt.vri_id = vd.vri_id
    WHERE lowerUTF8(vt.manufacturers) LIKE '%national%'
    
""")

In [15]:
full_df


,vri_id,verification_date,valid_date,org_title,mi_mitnumber,mi_mititle,mi_mitype,mi_modification,mi_number,manufacturers,mi_owner,vri_type,vri_type_label
0,2-151957054,2019-07-10 12:00:00,2020-07-09 12:00:00,"АО ""АКТИ-МАСТЕР""",74861-19,Преобразователи напряжения и силы тока цифро-а...,"NI 9263, NI 9264, NI 9265",,1DCBF04,"Компания ""National Instruments Corporation"", В...",,,
1,2-154475413,2019-08-02 12:00:00,2020-08-01 12:00:00,"АО ""АКТИ-МАСТЕР""",68422-17,Мультиметры цифровые модульные,"NI PXIe-4080, NI PXIe-4081, NI PXIe-4082",,1CCC913,"Компания ""National Instruments Malaysia Sdn. B...",,,
2,2-154475414,2019-08-02 12:00:00,2020-08-01 12:00:00,"АО ""АКТИ-МАСТЕР""",68422-17,Мультиметры цифровые модульные,"NI PXIe-4080, NI PXIe-4081, NI PXIe-4082",,1CB7EE2,"Компания ""National Instruments Malaysia Sdn. B...",,,
3,2-154475415,2019-08-02 12:00:00,2021-08-01 12:00:00,"АО ""АКТИ-МАСТЕР""",56624-14,Источники питания программируемые модульные,"NI PXIe-4112, NI PXIe-4113",,1CCF7A3,"Компания ""National Instruments Corporation"", В...",,,
4,2-154475416,2019-08-02 12:00:00,2021-08-01 12:00:00,"АО ""АКТИ-МАСТЕР""",56624-14,Источники питания программируемые модульные,"NI PXIe-4112, NI PXIe-4113",,1CE021D,"Компания ""National Instruments Corporation"", В...",,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,1-471273314,2025-09-29 00:00:00,2026-09-28 00:00:00,"АО ""НПО ""ЭЛЕКТРОМАШИНА""",78144-20,Преобразователи напряжения измерительные анало...,"NI 6221, NI 6251, NI 6255, NI 6259, NI 6281, N...",NI 6251,160FCDC,"Компания ""National Instruments Corporation"", В...","АО""НПО""Электромашина""",2,Периодическая
12192,1-471464687,2025-09-02 00:00:00,2026-09-01 00:00:00,"АО ""НПП ""РАДАР ММС""",74860-19,"Преобразователи напряжения, силы тока и сопрот...","NI 9201, NI 9203, NI 9205, NI 9215, NI 9219, N...",NI 9205,1CB23F8,"Компания ""National Instruments Corporation"", В...",Юридическое лицо,1,Первичная
12193,1-471464701,2025-09-02 00:00:00,2026-09-01 00:00:00,"АО ""НПП ""РАДАР ММС""",59845-15,Преобразователи напряжения и силы тока измерит...,"NI9208, NI9229, NI9239",NI 9208,20D9E5D,"Компания ""National Instruments Corporation"", В...",Юридическое лицо,1,Первичная
12194,1-471464710,2025-09-02 00:00:00,2026-09-01 00:00:00,"АО ""НПП ""РАДАР ММС""",74860-19,"Преобразователи напряжения, силы тока и сопрот...","NI 9201, NI 9203, NI 9205, NI 9215, NI 9219, N...",NI 9234,1E7D12B,"Компания ""National Instruments Corporation"", В...",Юридическое лицо,1,Первичная


In [16]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12196 entries, 0 to 12195
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   vri_id             12196 non-null  object        
 1   verification_date  12196 non-null  datetime64[ns]
 2   valid_date         12196 non-null  datetime64[ns]
 3   org_title          12196 non-null  object        
 4   mi_mitnumber       12196 non-null  object        
 5   mi_mititle         12196 non-null  object        
 6   mi_mitype          12196 non-null  object        
 7   mi_modification    12196 non-null  object        
 8   mi_number          12196 non-null  object        
 9   manufacturers      12196 non-null  object        
 10  mi_owner           12196 non-null  object        
 11  vri_type           12196 non-null  object        
 12  vri_type_label     12196 non-null  object        
dtypes: datetime64[ns](2), object(11)
memory usage: 1.2+ MB


In [17]:
full_df['mi_owner'].value_counts().head(20)

mi_owner
                                                                                 2459
Владелец                                                                         1358
Юридическое лицо                                                                 1246
ООО «Модульные Измерительные Решения»                                             277
АО "ВПК "НПО машиностроения"                                                      274
АО "ОКБ МЭЛ"                                                                      271
ООО "Фестон"                                                                      265
КБхиммаш                                                                          264
владелец                                                                          245
АО "ДИЗАЙН ЦЕНТР "СОЮЗ"                                                           242
АО "ЭНПО СПЭЛС"                                                                   241
-                                            

In [18]:
groups = full_df.groupby(["mi_modification", "mi_number"], sort=True)


In [19]:
sample = groups.get_group(("NI PXIe-4137", "2109ABB")).sort_values("verification_date")


In [20]:
summary = (
    groups["verification_date"]
    .agg(["min", "max", "count"])
    .reset_index()
    .rename(columns={"min": "first_verif", "max": "last_verif", "count": "checks"})
)

In [21]:
summary

,mi_modification,mi_number,first_verif,last_verif,checks
0,,00F11318,2019-04-02 12:00:00,2019-04-02 12:00:00,1
1,,00F14CD9,2020-03-17 12:00:00,2020-03-17 12:00:00,1
2,,00F15097,2019-05-06 12:00:00,2019-05-06 12:00:00,1
3,,00F161B4,2019-12-16 12:00:00,2019-12-16 12:00:00,1
4,,00F1DCDF,2019-05-06 12:00:00,2019-05-06 12:00:00,1
...,...,...,...,...,...
6645,нет модификации,F3CB88,2020-08-14 00:00:00,2020-08-14 00:00:00,1
6646,нет модификации,P15050075,2020-08-14 00:00:00,2020-08-14 00:00:00,1
6647,нет модификации,P16130166,2020-09-07 00:00:00,2020-09-07 00:00:00,1
6648,нет модификации,V130021005,2020-10-22 00:00:00,2020-10-22 00:00:00,1


In [22]:
groups = full_df.groupby(["mi_number"], sort=True)

In [23]:
summary = (
    groups["verification_date"]
    .agg(["min", "max", "count"])
    .reset_index()
    .rename(columns={"min": "first_verif", "max": "last_verif", "count": "checks"})
)

In [24]:
summary 


,mi_number,first_verif,last_verif,checks
0,00-000047,2023-01-23 00:00:00,2024-04-01 00:00:00,2
1,00-00047,2025-04-22 00:00:00,2025-04-22 00:00:00,1
2,001CBCF41,2020-08-18 00:00:00,2020-08-18 00:00:00,1
3,00DF91F0,2017-09-18 16:26:48,2017-09-18 16:26:48,1
4,00E07707,2020-12-10 00:00:00,2025-09-29 00:00:00,6
...,...,...,...,...
5270,б/н,2013-05-16 04:00:00,2018-06-26 03:00:00,4
5271,б/н - б/н,2017-10-17 03:00:00,2018-05-30 03:00:00,5
5272,р12280203,2024-07-25 00:00:00,2024-07-25 00:00:00,1
5273,хв001252,2023-08-11 00:00:00,2023-08-11 00:00:00,1


In [25]:
summary['checks'].mean()

np.float64(2.312037914691943)

In [26]:
mask = (
    full_df["mi_number"].fillna("")
    .str.strip()
    .str.replace(r"[^0-9A-Za-z]", "", regex=True)  # необязательный шаг, если хотите считать только буквы/цифры
    .str.len()
    .between(6, 8)
)

In [27]:
filtered_df = full_df[mask].copy()


In [28]:
summary = (
    filtered_df
    .groupby("mi_number")["verification_date"]
    .agg(["min", "max", "count"])
    .reset_index()
    .rename(columns={"min": "first_verif", "max": "last_verif", "count": "checks"})
)

Посмотрим первичные поверки

In [29]:
summary

,mi_number,first_verif,last_verif,checks
0,00-000047,2023-01-23 00:00:00,2024-04-01 00:00:00,2
1,00-00047,2025-04-22 00:00:00,2025-04-22 00:00:00,1
2,00DF91F0,2017-09-18 16:26:48,2017-09-18 16:26:48,1
3,00E07707,2020-12-10 00:00:00,2025-09-29 00:00:00,6
4,00E0771F,2020-12-11 00:00:00,2025-09-29 00:00:00,6
...,...,...,...,...
3727,ХВ001260,2023-08-21 00:00:00,2023-08-21 00:00:00,1
3728,ХВ001262,2023-08-21 00:00:00,2023-08-21 00:00:00,1
3729,р12280203,2024-07-25 00:00:00,2024-07-25 00:00:00,1
3730,хв001252,2023-08-11 00:00:00,2023-08-11 00:00:00,1


In [30]:
first_df = full_df.query('vri_type == "1"') 

In [31]:
first_df

,vri_id,verification_date,valid_date,org_title,mi_mitnumber,mi_mititle,mi_mitype,mi_modification,mi_number,manufacturers,mi_owner,vri_type,vri_type_label
1518,1-32369743,2020-12-23,2021-12-22,"АО ""АКТИ-МАСТЕР""",68422-17,Мультиметры цифровые модульные,"NI PXIe-4080, NI PXIe-4081, NI PXIe-4082",PXIe-4081,1F43C05,"Компания ""National Instruments Malaysia Sdn. B...","ООО ""НТЦ ПРОМИКА""",1,Первичная
1519,1-32369744,2020-12-23,2022-12-22,"АО ""АКТИ-МАСТЕР""",72912-18,Источники питания программируемые,RMX-412x,RMX-4122,AY000056,"Фирма ""National Instruments Corporation"", Венг...","ООО ""НТЦ ПРОМИКА""",1,Первичная
1520,1-32369745,2020-12-23,2022-12-22,"АО ""АКТИ-МАСТЕР""",72912-18,Источники питания программируемые,RMX-412x,RMX-4122,WM002434,"Фирма ""National Instruments Corporation"", Венг...","ООО ""НТЦ ПРОМИКА""",1,Первичная
1521,1-32369746,2020-12-23,2021-12-22,"АО ""АКТИ-МАСТЕР""",75842-19,Генераторы сигналов произвольной формы модульные,"NI PXIe-5413, NI PXIe-5423, NI PXIe-5433",PXIe-5413,1F49497,"Компания ""National Instruments Corporation"", В...","ООО ""НТЦ ПРОМИКА""",1,Первичная
1522,1-32369747,2020-12-23,2021-12-22,"АО ""АКТИ-МАСТЕР""",68422-17,Мультиметры цифровые модульные,"NI PXIe-4080, NI PXIe-4081, NI PXIe-4082",PXIe-4081,1F33DE9,"Компания ""National Instruments Malaysia Sdn. B...","ООО ""НТЦ ПРОМИКА""",1,Первичная
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12132,1-467926350,2025-09-25,2026-09-24,"АО ""АКТИ-МАСТЕР""",68422-17,Мультиметры цифровые модульные,"NI PXIe-4080, NI PXIe-4081, NI PXIe-4082",NI PXIe-4081,1EC5B19,"Компания ""National Instruments Malaysia Sdn. B...",Юридическое лицо,1,Первичная
12192,1-471464687,2025-09-02,2026-09-01,"АО ""НПП ""РАДАР ММС""",74860-19,"Преобразователи напряжения, силы тока и сопрот...","NI 9201, NI 9203, NI 9205, NI 9215, NI 9219, N...",NI 9205,1CB23F8,"Компания ""National Instruments Corporation"", В...",Юридическое лицо,1,Первичная
12193,1-471464701,2025-09-02,2026-09-01,"АО ""НПП ""РАДАР ММС""",59845-15,Преобразователи напряжения и силы тока измерит...,"NI9208, NI9229, NI9239",NI 9208,20D9E5D,"Компания ""National Instruments Corporation"", В...",Юридическое лицо,1,Первичная
12194,1-471464710,2025-09-02,2026-09-01,"АО ""НПП ""РАДАР ММС""",74860-19,"Преобразователи напряжения, силы тока и сопрот...","NI 9201, NI 9203, NI 9205, NI 9215, NI 9219, N...",NI 9234,1E7D12B,"Компания ""National Instruments Corporation"", В...",Юридическое лицо,1,Первичная


In [29]:
full_df['vri_type'].value_counts()

vri_type
2    7867
     2662
1    1667
Name: count, dtype: int64

In [33]:
first_df['mi_mitnumber'].value_counts().head(25)

mi_mitnumber
47898-11    319
74860-19    165
78142-20    116
78144-20     85
68422-17     81
76139-19     77
75842-19     59
61939-15     57
75841-19     51
57582-14     46
48245-11     42
58297-14     34
68973-17     31
57392-14     30
53913-13     29
71361-18     29
72912-18     29
61391-15     27
79181-20     27
74861-19     25
69739-17     22
48201-11     22
50805-12     21
57391-14     17
76642-19     17
Name: count, dtype: int64

In [35]:
first_df['mi_owner'].value_counts().head(25)

mi_owner
Юридическое лицо                                                                                                       336
ООО "Фестон"                                                                                                           261
ООО «Модульные Измерительные Решения»                                                                                  165
ООО «ВИП Технологии»                                                                                                   128
ООО "МАКСПРОМ"                                                                                                         104
ООО "НТЦ ПРОМИКА"                                                                                                       62
Владелец                                                                                                                59
АО "ДИЗАЙН ЦЕНТР "СОЮЗ"                                                                                                 35
Юр. Лиц

In [37]:
vxi = sql("""
SELECT *
FROM v_vri_with_type
WHERE manufacturers LIKE 'VXI%'
AND match(mi_mitnumber, '^(95442|95403|94918|94247|90029|90006|88767|87478|86719|86357|84676|84581|84413)')
""")

In [38]:
vxi

""
